In [245]:
import numpy as np
import pandas as pd
import sqlalchemy
from sqlalchemy.engine import create_engine
import pymysql

In [246]:
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql import SparkSession

In [247]:
## preto读数据库
def create_presto_engine():
    conn = 'presto://reed-algo-offline-01:20080'
    engine = create_engine(conn, poolclass=sqlalchemy.pool.NullPool)  #--connect_args={'username': null, 'password': null, 'group': null}
    return engine
engine = create_presto_engine()

def read_hive(sql, pd_df_col=[]):  #--可选参数:df列名
    # sql_ins = sql.format(date_y_m_d='1970-01-01', date='19700101')
    # ndata = pd.read_sql(sql_ins, engine)
    ndata = pd.read_sql(sql, engine)
    if pd_df_col != []:
        ndata.columns = pd_df_col
    return ndata


In [ ]:
# 随机分组
mart_reed_algo_ad.grouplive_algo_expcontrol_grouping_i_d
# 算法分组
mart_reed_algo_ad.grouplive_algo_expcontrol_subgrouping_i_d

要把这个给考虑上

In [249]:
from datetime import datetime, timedelta
living_dt = '2025-04-09'
living_dt1 = (datetime.strptime(living_dt, '%Y-%m-%d') - timedelta(days=2)).strftime('%Y-%m-%d')
living_dt7 = (datetime.strptime(living_dt, '%Y-%m-%d') - timedelta(days=7)).strftime('%Y-%m-%d')

## 读数

In [250]:
### 数仓分组
df_unionid = read_hive('''
select t1.unionid, t1.dep_id,project_c_tag, is_friend, t1.invitor_userid, t1.dt, t2.shop_type
FROM dws.dws_activity_brain_user_group_detail_result_1d_d t1
left join dwd.dwd_user_user_detail_a_d t2
on t1.unionid = t2.unionid
WHERE t1.dt = (SELECT MAX(dt) FROM dws.dws_activity_brain_user_group_detail_result_1d_d)
''')

In [251]:
df_unionid.shape

(3720137, 7)

In [252]:
df_unionid.unionid.nunique()

3720137

In [254]:
### 跟线统计

df_unionid_followline = read_hive(f'''

-- 团品直播id
with tmp_livingid as 
(select 
cast(id as varchar) as living_id,
cast(living_duration as double) as living_duration,
market_scope,
substr(living_start, 1, 10) as dt
from 
dwd.dwd_live_polyv_living_info_a_d
where 
(
    (living_start >= '2025-03-03' and living_start <= '2025-03-20'
    and viewer_num > 80000
    and substr(living_start,12,2) in ('08','09','10','11')) 
    or
    (living_start >= '2025-03-21' 
    and viewer_num > 10000
    and substr(living_start,12,2) in ('08','09','10','11')
    and has_command_center = 1 
    and status = 2 and use_type = 1 
    )
)
 and (substr(living_start,1,10) between '{living_dt7}' and '{living_dt1}')
),

-- 参加直播的部门
tmp_dep as (
select 
dep_id, count(distinct unionid) as living_user_cnt
from
(select t2.unionid, dt, livingid, dep_id from
(select unionid,livingid
from dwd.dwd_live_living_stat_all_i_d where dt between '{living_dt7}' and '{living_dt1}' and unionid is not null) t2 
inner join tmp_livingid t3
on t2.livingid = t3.living_id
left join 
(select unionid, dep_id  from dwd.dwd_user_user_detail_a_d  where unionid is not null) t4
on t2.unionid = t4.unionid )
group by dep_id
),

-- 部门过滤(去掉参加人数较少的门店，认为不是跟线门店)
tmp_dep_filter as (
select 
t1.dep_id
--, living_user_cnt,total_user_cnt, cast(living_user_cnt as double)/cast(total_user_cnt as double) as living_rate
from 
(select dep_id, living_user_cnt from tmp_dep) t1
left join (select dep_id,count(distinct unionid) as total_user_cnt  from dws.dws_activity_brain_user_group_detail_result_1d_d where dt = (SELECT MAX(dt) FROM dws.dws_activity_brain_user_group_detail_result_1d_d)
group by dep_id) t2
on t1.dep_id = t2.dep_id
where cast(living_user_cnt as double)/cast(total_user_cnt as double) >= 0.01 and living_user_cnt > 10
),

-- 非沉默用户
tmp_unionid_active as (
select unionid,dep_id from dwd.dwd_user_user_detail_a_d 
where cast(substr(video_add_time,1,10) as date) >= date_add('day', -30, date('{living_dt1}'))
union all 
select unionid,depid dep_id
from dwd.dwd_live_living_stat_all_i_d 
where dt between cast(date_add('day', -90, date('{living_dt1}')) as varchar) and '{living_dt1}'
group by unionid,depid
)

-- 跟线非沉默用户
select unionid, 1 as flag_followline 
from tmp_unionid_active t1 inner join tmp_dep_filter t2 on t1.dep_id = t2.dep_id
group by unionid
''')

In [253]:
df_unionid_followline.shape

(957151, 2)

In [279]:
### 跟线合并
df_unionid = df_unionid.merge(df_unionid_followline, on = 'unionid', how='left')

## 算法逻辑（目前为全推，后续要迭代）

In [280]:
### 算法分组
np.random.seed(0)
# 1. 对 project_c_tag = 'A' 的行随机分配 'exp1' 或 'exp2'
mask_A = df_unionid['project_c_tag'] == 'A'
df_unionid.loc[mask_A, 'project_c_subtag'] = np.random.choice(
    ['exp1', 'exp2'], 
    size=mask_A.sum(),
    p=[1, 0]
)
# 2. 其余行用 project_c_tag 填充
df_unionid.loc[~mask_A, 'project_c_subtag'] = df_unionid.loc[~mask_A, 'project_c_tag']

In [281]:
### 模型排序结果
df_unionid_modelrank = read_hive('''
select unionid, prob_visit as model_prob from mart_reed_algo_ad.grouplive_model_yearshopping_pred_res 
''')
df_unionid = df_unionid.merge(df_unionid_modelrank, on='unionid', how='left')
df_unionid["prob_rank"] = df_unionid.groupby("invitor_userid")["model_prob"].rank(method="first", ascending=False)

In [183]:
print(df_unionid.shape)
print(df_unionid.unionid.nunique())

(3720137, 11)
3720137


In [282]:
### 预热、私聊打标
# 预热打标
df_unionid['flag_warmup'] = np.where(
    (df_unionid['flag_followline'] == 1) & 
    (df_unionid['project_c_subtag'] == 'exp1') & 
    (df_unionid['shop_type'].isin([1, 2])),  
    1, 0   
)

# 私聊打标
df_unionid['flag_chat'] = np.where(
    (df_unionid['flag_followline'] == 1) & 
    (df_unionid['project_c_subtag'] == 'exp1') & 
    (df_unionid['is_friend'] == 1) & 
    (df_unionid['shop_type'].isin([1, 2])), 
    1, 0   
)

In [186]:
# ### 存表
# df_unionid['partition_insert'] = pd.qcut(df_unionid.index, q=40, labels=False)
# df_unionid = df_unionid[['unionid', 'group_name', 'subgroup_name','shop_type','flag_followline','qx_userid','partition_insert','dt_group',
# 'live_cnt_count_7', 'video_action3_cnt_count_7','live_cnt_count_30', 'video_action3_cnt_count_30' ,'live_cnt_count_90', 'video_action3_cnt_count_90','dt_subgroup']]
# from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType,FloatType
# import time
# schema = StructType([
#     StructField("unionid", StringType(), True),
#     StructField("group_name", StringType(), True),
#     StructField("subgroup_name", StringType(), True),
#     StructField("shop_type", FloatType(), True),
#     StructField("flag_followline", FloatType(), True),  
#     StructField("qx_userid", StringType(), True),
#     StructField("partition_insert", IntegerType(), True),
#     StructField("dt_group", StringType(), True),    
#     StructField("live_cnt_count_7", FloatType(), True),
#     StructField("video_action3_cnt_count_7", FloatType(), True),    
#     StructField("live_cnt_count_30", FloatType(), True),
#     StructField("video_action3_cnt_count_30", FloatType(), True),
#     StructField("live_cnt_count_90", FloatType(), True),
#     StructField("video_action3_cnt_count_90", FloatType(), True),
#     StructField("dt_subgroup", StringType(), True)
# ])
# grouped_df = df_data_split.groupby('partition_insert').size().reset_index(name='num')
# spark = SparkSession.builder.appName('grouplive_app').enableHiveSupport()\
#     .config("hive.exec.dynamic.partition.mode", "nonstrict")\
#     .getOrCreate()
# for partition_insert in grouped_df['partition_insert']:
#     # if partition_insert % 5 == 0:
#     print(partition_insert)
#     merged_df2=df_data_split[(df_data_split['partition_insert'] == partition_insert)]
#     spark_df_model_result= spark.createDataFrame(merged_df2, schema=schema)
#     spark_df_model_result.write.mode('append').format('hive').partitionBy('dt_subgroup').saveAsTable('mart_reed_algo_ad.grouplive_algo_expcontrol_subgrouping_i_d')
# spark.stop()

In [187]:
# 创建目录存放文件
# ! mkdir /data1/zhangzhen/grouplive/2025-04-04/

mkdir: cannot create directory ‘/data1/zhangzhen/grouplive/2025-04-04/’: File exists


In [188]:
# df_unionid.to_csv('/data1/zhangzhen/grouplive/2025-04-04/df_unionid_res.csv', index=False)

In [ ]:
### 2025-04-02

In [234]:
# df_unionid = pd.read_csv('/data1/zhangzhen/grouplive/2025-03-31/df_unionid_res.csv')

## 预热

In [259]:
living_dt = '2025-04-06'
# 线上读直播间
connection = pymysql.connect(host='pc-2ze5hgvv9n4uic10t.rwlb.rds.aliyuncs.com', port=3306,  user='zhangzhen', password='zhangzhen@53567', database='thrk', )
with connection.cursor() as cursor:
    sql = f'''
    select t1.living_id, living_name, living_start, market_scope, shop_id from
    (select id as living_id, name as living_name,living_start, market_scope
    from thrk.polyv_living_info where substr(living_start,1,10) >= '{living_dt}' and substr(living_start,1,10) <= '{living_dt}' and status = 0 and has_command_center = 1 and use_type = 1 and dep_id = 955
    ) t1
    left join 
    (select living_id, shop_id from thrk.polyv_living_shop) t2
    on t1.living_id = t2.living_id

    '''
    cursor.execute(sql)
    result = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]

    df_living = pd.DataFrame(result, columns=columns)
connection.close()


In [266]:
df_living = df_living.query('shop_id == shop_id')

In [267]:
shop_ids = df_living['shop_id'].unique().tolist()
shop_ids_str = ",".join(map(str, shop_ids))

In [268]:
# dep_id 对应

df_shop_dep = read_hive(f'''
SELECT
      if(d1.parentid IN ({shop_ids_str}), d1.parentid,    
        if(d2.parentid IN ({shop_ids_str}), d2.parentid,
         if(d3.parentid IN ({shop_ids_str}), d3.parentid, 
          if(d4.parentid IN ({shop_ids_str}), d4.parentid, null)))) as shop_id,
    d1.id dep_id
FROM dim.dim_pub_qx_epartment d1
LEFT JOIN dim.dim_pub_qx_epartment d2 ON d1.parentid = d2.id
LEFT JOIN dim.dim_pub_qx_epartment d3 ON d2.parentid = d3.id
LEFT JOIN dim.dim_pub_qx_epartment d4 ON d3.parentid = d4.id
LEFT JOIN dim.dim_pub_qx_epartment d5 ON d4.parentid = d5.id
WHERE (
    d1.parentid IN ({shop_ids_str})
    OR d2.parentid IN ({shop_ids_str})
    OR d3.parentid IN ({shop_ids_str})
    OR d4.parentid IN ({shop_ids_str})
)
AND COALESCE(d1.is_del, 0) = 0
AND d1.delete_time IS NULL
''')

In [270]:
# 直播间部门关联
df_living = df_living.merge(df_shop_dep, on='shop_id', how='left')

In [272]:
# 预热视频
connection = pymysql.connect(host='pc-2ze5hgvv9n4uic10t.rwlb.rds.aliyuncs.com', port=3306,  user='zhangzhen', password='zhangzhen@53567', database='thrk', )
with connection.cursor() as cursor:
    sql = f'''
    select t1.living_id, t3.video_id, label_id,video_duration from
    (select id as living_id, name as living_name,living_start, market_scope
    from thrk.polyv_living_info where substr(living_start,1,10) >= '{living_dt}' and substr(living_start,1,10) <= '{living_dt}' and status = 0 and has_command_center = 1 and use_type = 1 and dep_id = 955
    ) t1
    
    inner join
    (select id, polyv_id  from newplatform.polyv_subscribe_rule_config where valid = 1) t2
    on t1.living_id = t2.polyv_id
    
    left join 
    (select subscribe_id, video_id from newplatform.polyv_video_relation where valid = 1) t3
    on t2.id = t3.subscribe_id
    
    left join
    -- 504-合集 505-单品
    (select video_id, label_id from newplatform.video_label_relation) t4
    on t3.video_id = t4.video_id   
    
    left join
    (select id, seconds as video_duration from newplatform.v2_rui_video where substr(create_time,1,10)>= '2025-03-21') t5
    on t3.video_id = t5.id      



    '''
    cursor.execute(sql)
    result = cursor.fetchall()
    columns = [desc[0] for desc in cursor.description]

    df_living_video = pd.DataFrame(result, columns=columns)
connection.close()


In [273]:
df_living_video

,living_id,video_id,label_id,video_duration
0,36786,111681,505,45
1,36786,111680,505,46
2,36788,111679,505,15


In [274]:
df_living_video = df_living_video.sort_values(by = ['label_id', 'video_duration'], ascending=[True, False])
df_living_video = df_living_video.merge(df_living, on='living_id', how='left')
from datetime import datetime, timedelta
def func_video(x):
    d = {}
    d['market_scope'] = x.iloc[0].market_scope
    d['video_format'] = ','.join([str(video_id) for video_id in x.video_id])
    previous_day = x.iloc[0].living_start -  timedelta(days=1)
    current_time_str = x.iloc[0].living_start.strftime('%Y%m%d%H%M')
    previous_time_str = previous_day.strftime('%Y%m%d%H%M')
    d['dt_format'] = f"{previous_time_str}_{current_time_str}"
    return pd.Series(d, index=d.keys())
df_living_warmup = df_living_video.groupby(['living_id', 'dep_id']).apply(func_video).reset_index()

In [275]:
df_living_warmup.head(5)

,living_id,dep_id,market_scope,video_format,dt_format
0,36786,2,2,"111680,111681",202504050755_202504060755
1,36786,4,2,"111680,111681",202504050755_202504060755
2,36786,5,2,"111680,111681",202504050755_202504060755
3,36786,6,2,"111680,111681",202504050755_202504060755
4,36786,20,2,"111680,111681",202504050755_202504060755


In [284]:
# 预热过滤
df_unionid_combind = df_unionid.merge(df_living_warmup,
    left_on=['dep_id', 'shop_type', ],  
    right_on=['dep_id', 'market_scope'],
    how='left')

In [285]:
df_warmup_filter = df_unionid_combind.query('flag_warmup == 1 and video_format == video_format')
df_warmup_filter['unionid_format'] = 'grouplive$' + df_warmup_filter['unionid'] + '$'
df_warmup_filter['unionid_dt_format'] = df_warmup_filter['unionid_format'] + df_warmup_filter['dt_format']
df_warmup_filter['unionid_dt_video_format'] = df_warmup_filter['video_format']

/usr/local/miniconda3/envs/jupyter_hub37/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/miniconda3/envs/jupyter_hub37/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/miniconda3/envs/jupyter_hub37/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a

In [286]:
df_warmup_filter.shape

(148539, 20)

In [199]:
# %%time
# import redis
# from datetime import timedelta
# BATCH_SIZE = 5000  # 每批次插入 1000 条数据
# client = redis.StrictRedis(host='r-2ze7b3wslfewb38p0i.redis.rds.aliyuncs.com', port=6379, db=1, password='devops:jKvbe%zx54&P')
# # 设置键的生命周期（TTL）
# TTL = timedelta(days=2)  # 设置键的生命周期为 7 天

# # 批次插入 key_map 和 key
# def batch_insert_keymap(df, batch_size):
#     pipeline = client.pipeline()
#     for index, row in df[['unionid_format', 'unionid_dt_format']].iterrows():
#         key = row['unionid_format']
#         val = row['unionid_dt_format']
#         pipeline.set(key, val, ex=TTL)  # 设置 TTL
#         if (index + 1) % batch_size == 0:  # 每批次执行一次
#             pipeline.execute()
#             pipeline = client.pipeline()  # 重置 pipeline
#     if len(pipeline) > 0:  # 处理剩余的记录
#         pipeline.execute()

# # 批次插入 key 和 val
# def batch_insert_keyval(df, batch_size):
#     pipeline = client.pipeline()
#     for index, row in df[['unionid_dt_format', 'unionid_dt_video_format']].iterrows():
#         key = row['unionid_dt_format']
#         val = row['unionid_dt_video_format']
#         pipeline.set(key, val, ex=TTL)  # 设置 TTL
#         if (index + 1) % batch_size == 0:  # 每批次执行一次
#             pipeline.execute()
#             pipeline = client.pipeline()  # 重置 pipeline
#     if len(pipeline) > 0:  # 处理剩余的记录
#         pipeline.execute()

# # 执行批次插入
# batch_insert_keymap(df_warmup_filter, BATCH_SIZE)
# batch_insert_keyval(df_warmup_filter, BATCH_SIZE)

CPU times: user 1min 7s, sys: 571 ms, total: 1min 8s
Wall time: 1min 30s


## 私聊

In [26]:
# # 无预热
# df_combind = df_unionid.query('flag_chat == 1')
# df_combind = df_combind.reset_index(drop=True)
# df_living_expanded = pd.concat([df_living] * len(df_combind), ignore_index=True)
# df_combind = pd.concat([df_combind, df_living_expanded], axis=1)

In [213]:
# 有预热
df_combind = df_unionid_combind.query('flag_chat == 1')
df_combind = df_combind.query('living_id == living_id')

In [218]:
df_combind.shape

(441332, 17)

In [219]:

## 合并为json content
def merge_text_and_picture(row):
    return [
        {
            "type": "text",
            "value": 
            '''激动的心，颤抖的手，人康好物在招手！早8点，锁定人康直播间，高品质配优价格，千万别错过！'''
        },
        {
            "type": "ext",
            "value": ""
        }
    ]
# 应用函数并创建新列
df_combind['content'] = df_combind.apply(merge_text_and_picture, axis=1)
df_combind = df_combind.rename(columns={
    'living_id': 'living_id',
    'invitor_userid': 'send_user_id',
    'unionid': 'union_id',
    'content': 'content',
    # 'living_start': 'send_time',
    'prob_rank': 'prob_rank'
})
df_combind['send_time'] = pd.to_datetime(f'{living_dt} 07:00:00')
df_combind = df_combind[['living_id', 'send_user_id', 'union_id', 'content', 'send_time', 'prob_rank']]
df_combind['content'] = df_combind['content'].apply(lambda x: str(x))
df_combind['prob_rank'] = df_combind['prob_rank'].fillna(0)

/usr/local/miniconda3/envs/jupyter_hub37/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [207]:
df_combind.head(1)

,living_id,send_user_id,union_id,content,send_time,prob_rank
6,36783.0,zhanghongbo_t55y,oPN705-gL2onejYdvbL1ZcXKTYuQ,"[{'type': 'text', 'value': '激动的心，颤抖的手，人康好物在招手！...",2025-04-04 07:00:00,91.0


In [220]:
# # 线上环境
connection = pymysql.connect(
    host='pc-2ze5hgvv9n4uic10t.rwlb.rds.aliyuncs.com',  # 使用 url 中的主机地址
    port=3306,  # 默认 MySQL 端口
    user='market',  # 使用 username
    password='market@123',  # 使用 password
    database='market_brain',  # 数据库名称
)
cursor = connection.cursor()
cursor.execute(f"TRUNCATE TABLE market_task_live_rec")
sql = "INSERT INTO market_task_live_rec (living_id,send_user_id,union_id,content,send_time,prob_rank) VALUES (%s, %s, %s, %s, %s, %s)"
data = [tuple(x) for x in df_combind.values]
cursor.executemany(sql, data)
connection.commit()
cursor.close()
connection.close()
print("插入mysql生产_分层成功",df_combind.shape)

插入mysql生产_分层成功 (441332, 6)


In [221]:
# import requests

# # 测试环境 URL
# test_url = "https://teststate.renruikeji.cn/api/marketbrain/sop/job/liveRecDataReady"

# # 生产环境 URL
# prod_url = "https://state.renruikeji.cn/api/marketbrain/sop/job/liveRecDataReady"

# # 选择环境（测试或生产）
# url = prod_url  # 或者 prod_url

# # 发送 GET 请求
# response = requests.get(url)

# # 检查响应状态码
# if response.status_code == 200:
#     print("请求成功！")
#     # print("响应内容：", response.json())  # 如果返回的是 JSON 数据
# else:
#     print(f"请求失败，状态码：{response.status_code}")
#     # print("错误信息：", response.text)

请求成功！
